Code to import WRDS data to PostgreSQL
=========

This repository contains code to pull together data from [WRDS](https://wrds-web.wharton.upenn.edu/wrds/).

The code will only work if you have access to WRDS and to the data in question.

## Requirements

### 1. Git

While not strictly necessary to use the scripts here, [Git](https://git-scm.com/downloads) likely makes it easier to download and to update.

I keep all Git repositories in `~/git`. So to get this repository, I could do:

```
cd ~/git
git clone https://github.com/iangow/wrds_pg.git
```

This will create a copy of the repository in `~/git/wrds_pg`.
Note that one can get updates to the repository by going to the directory and "pulling" the latest code:

```
cd ~/git/wrds_pg
git pull
```

Alternatively, I think you could fork the repository on GitHub and then clone. 
I think that cloning using the SSH URL (e.g., `git@github.com:iangow/wrds_pg.git`) is necessary for Git pulling and pushing to work well in RStudio.

### 2. Python

The code has been tested on Python 3 and requires the following packages:

- `paramiko`: for SSH communication with the WRDS server.
- `pandas`: for various utility purposes (would be easy to remove this dependency, but it is easily satisfied).
- `sqlalchemy`: for database-related stuff.
- `psycopg2`: Used by SQLAlchemy to connect to PostgreSQL.

### 2. PostgreSQL

You should have a PostgreSQL database to store the data.
There are also some data dependencies in that some scripts assume the existence of other data in the database.
Also, I assume the existence of role `wrds` and `wrds_access` (SQL like `CREATE ROLE wrds` works to add these if they are absent).

### 3. Environment variables

I am migrating the scripts, etc., from using hard-coded values (e.g., my WRDS ID `iangow`) to using environment variales. 
Environment variables that I use include:

- `PGDATABASE`: The name of the PostgreSQL database you use.
- `PGUSER`: Your username on the PostgreSQL database.
- `PGHOST`: Where the PostgreSQL database is to be found (this will be `localhost` if its on the same machine as you're running the code on)
- `WRDS_ID`: Your [WRDS](https://wrds-web.wharton.upenn.edu/wrds/) ID.

I set these environment variables in `~/.profile`:

```
export PGHOST="localhost"
export PGDATABASE="crsp"
export WRDS_ID="iangow"
export PGUSER="igow"
```

### 4. A WRDS ID

Note that I use public-key authentication to access WRDS. Following hints taken from [here](http://www.debian-administration.org/articles/152), I set up a public key. I then copied that key to the WRDS server from the terminal on my computer. (Note that this code assumes you have a directory `.ssh` in your home directory. If not, log into WRDS via SSH, then type `mkdir ~/.ssh` to create this.) Here's code to create the key and send it to WRDS (for me):

```
ssh-keygen -t rsa
cat ~/.ssh/id_rsa.pub | ssh iangow@wrds-cloud.wharton.upenn.edu "cat >> ~/.ssh/authorized_keys"
```

I use an empty passphrase in setting up my key so that the scripts can run without user intervention.

In [1]:
from sqlalchemy import create_engine
import os
dbname = "crsp" # os.getenv("PGDATABASE")
host = "iangow.me" # os.getenv("PGHOST", "localhost")
user = os.getenv("PGUSER", os.getlogin())
wrds_id = "iangow" # os.getenv("WRDS_ID")
dbname = engine = create_engine("postgresql://" + user + "@" + host + "/" + dbname)

In [2]:
from wrds_fetch import get_table_sql
df = get_table_sql("msf", "crsp", wrds_id, return_sql=False)
df

,libname,memname,memlabel,typemem,name,type,length,varnum,label,format,...,collate,nodupkey,noduprec,encrypt,pointobs,genmax,gennum,gennext,transcod,postgres_type
0,CRSP,MSF,Monthly Stock - Securities,NaN,cusip,2,8,1,CUSIP Header,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,text
1,CRSP,MSF,Monthly Stock - Securities,NaN,permno,1,8,2,PERMNO,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,int8
2,CRSP,MSF,Monthly Stock - Securities,NaN,permco,1,8,3,PERMCO,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,int8
3,CRSP,MSF,Monthly Stock - Securities,NaN,issuno,1,8,4,Nasdaq Issue Number,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,int8
4,CRSP,MSF,Monthly Stock - Securities,NaN,hexcd,1,8,5,Exchange Code Header,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,int8
5,CRSP,MSF,Monthly Stock - Securities,NaN,hsiccd,1,8,6,Standard Industrial Classification Code,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,int8
6,CRSP,MSF,Monthly Stock - Securities,NaN,date,1,8,7,Date of Observation,YYMMDDN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,date
7,CRSP,MSF,Monthly Stock - Securities,NaN,bidlo,1,8,8,Bid or Low Price,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,float8
8,CRSP,MSF,Monthly Stock - Securities,NaN,askhi,1,8,9,Ask or High Price,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,float8
9,CRSP,MSF,Monthly Stock - Securities,NaN,prc,1,8,10,Price or Bid/Ask Average,NaN,...,NaN,YES,NO,NO,YES,0,NaN,NaN,YES,float8


In [35]:
df['name']

0        cusip
1       permno
2       permco
3       issuno
4        hexcd
5       hsiccd
6         date
7        bidlo
8        askhi
9          prc
10         vol
11         ret
12         bid
13         ask
14      shrout
15      cfacpr
16     cfacshr
17      altprc
18      spread
19    altprcdt
20        retx
Name: name, dtype: object

In [37]:
df.loc[:,['name', 'type', 'length', 'postgres_type', 'format', 'formatl', 'formatd', 'informat', 'informl', 'informd']]

,name,type,length,postgres_type,format,formatl,formatd,informat,informl,informd
0,cusip,2,8,text,NaN,8,0,NaN,8,0
1,permno,1,8,int8,NaN,8,0,NaN,8,0
2,permco,1,8,int8,NaN,8,0,NaN,8,0
3,issuno,1,8,int8,NaN,8,0,NaN,8,0
4,hexcd,1,8,int8,NaN,2,0,NaN,2,0
5,hsiccd,1,8,int8,NaN,8,0,NaN,8,0
6,date,1,8,date,YYMMDDN,8,0,NaN,0,0
7,bidlo,1,8,float8,NaN,12,5,NaN,12,5
8,askhi,1,8,float8,NaN,12,5,NaN,12,5
9,prc,1,8,float8,NaN,12,5,NaN,12,5


In [19]:
df.dtypes

libname           object
memname           object
memlabel          object
typemem          float64
name              object
type               int64
length             int64
varnum             int64
label             object
format            object
formatl            int64
formatd            int64
informat          object
informl            int64
informd            int64
just               int64
npos               int64
nobs               int64
engine            object
crdate            object
modate            object
delobs             int64
idxusage          object
memtype           object
idxcount           int64
protect           object
flags             object
compress          object
reuse             object
sorted             int64
sortedby         float64
charset           object
collate          float64
nodupkey          object
noduprec          object
encrypt           object
pointobs          object
genmax             int64
gennum           float64
gennext          float64


In [1]:
from wrds_fetch import wrds_update, set_table_comment
print(wrds_id)

iangow


In [2]:
schema = "crsp"
table_name = "dsi"

In [3]:
from wrds_fetch import wrds_update
wrds_update(table_name, schema, engine, wrds_id, force=True)

Forcing update based on user request.


True

In [4]:
import pandas as pd
df = pd.read_sql("""SELECT * FROM "%s"."%s" LIMIT 6""" % (schema, table_name), engine)
df

,date,vwretd,vwretx,ewretd,ewretx,sprtrn,spindx,totval,totcnt,usdval,usdcnt
0,1925-12-31,NaN,NaN,NaN,NaN,None,None,27487487.2,503,NaN,NaN
1,1926-01-02,0.005689,0.005689,0.009516,0.009516,None,None,27600296.6,497,27366755.7,494.0
2,1926-01-04,0.000706,0.000706,0.005780,0.005780,None,None,27577892.1,502,27479721.6,495.0
3,1926-01-05,-0.004821,-0.004867,-0.001927,-0.002030,None,None,27530212.5,501,27561757.1,499.0
4,1926-01-06,-0.000423,-0.000427,0.001182,0.001155,None,None,27618772.6,505,27526837.5,500.0
5,1926-01-07,0.004988,0.004953,0.008453,0.008384,None,None,27690393.8,504,27553072.6,504.0
